In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



## Import libraries[](http://)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
#from sklearn.externals.six import StringIO
from sklearn import tree
from six import StringIO
from sklearn.model_selection import KFold, cross_val_score
from sklearn import metrics


#visualization
%matplotlib inline
import seaborn as sns
from IPython.display import Image
#import pydotplus
from sklearn.tree import export_graphviz

## EDA

In [ ]:
df = pd.read_csv('../input/arketing-campaign/marketing_campaign.csv', delimiter=';')

In [ ]:
df.head()

In [ ]:
df_cleaned = df.copy()

del df_cleaned['Z_CostContact']
del df_cleaned['Z_Revenue']


In [ ]:
#create account age
df_cleaned['Dt_Customer'] = pd.to_datetime(df_cleaned['Dt_Customer'])
df_cleaned['Dt_Customer_year_month'] = df_cleaned['Dt_Customer'].dt.to_period("M")
df_cleaned['account_age'] = (pd.to_datetime("2014-12").year - df_cleaned['Dt_Customer_year_month'].dt.year)*12 + (pd.to_datetime("2014-12").month - df_cleaned['Dt_Customer_year_month'].dt.month)

In [ ]:
del df_cleaned['Dt_Customer_year_month']
del df_cleaned['Dt_Customer']
df_cleaned.head()

In [ ]:
# create customer age
df_cleaned['Age'] = 2014 - df_cleaned["Year_Birth"]
del df_cleaned['Year_Birth']
df_cleaned.head()

In [ ]:
df_cleaned.describe()

In [ ]:
# Fillna in Income by using avg value

avg_income = np.mean(df_cleaned.Income)

df_cleaned['Income'] = df_cleaned['Income'].fillna(avg_income, axis = 0)

In [ ]:
df_cleaned.describe()

In [ ]:
df_cleaned.corr()

In [ ]:
#from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#Create Dummy Variables
enc_edu = OneHotEncoder(handle_unknown='ignore')
enc_edu_df = pd.DataFrame(enc_edu.fit_transform(df_cleaned[['Education']]).toarray())

df_cleaned = df_cleaned.join(enc_edu_df)

df_cleaned.head()

In [ ]:
df_cleaned.rename(columns = {0: "2n Cycle", 1: "Basic", 2: "Graduation", 3: "Master", 4: "PhD"},inplace = True)

In [ ]:
enc_marital = OneHotEncoder(handle_unknown='ignore')
enc_marital_df = pd.DataFrame(enc_edu.fit_transform(df_cleaned[['Marital_Status']]).toarray())

df_cleaned = df_cleaned.join(enc_marital_df)

df_cleaned.rename(columns = {0: "Absurd", 1: "Alone", 2: "Divorced", 3: "Married", 4: "Single", 5: "Together", 6: "Widow", 7: "YOLO"}, inplace = True)
df_cleaned.head()

In [ ]:
del df_cleaned['Marital_Status']
del df_cleaned['Education']

In [ ]:
df_cleaned["total_Mnt"] = df_cleaned["MntWines"] + df_cleaned["MntFruits"] + df_cleaned["MntMeatProducts"]+ df_cleaned['MntFishProducts'] + df_cleaned["MntSweetProducts"] + df_cleaned["MntGoldProds"]

df_cleaned['MntWines_pct'] = df_cleaned['MntWines']/df_cleaned['total_Mnt']
df_cleaned['MntFruits_pct'] = df_cleaned["MntFruits"]/df_cleaned['total_Mnt']
df_cleaned["MntMeatProducts_pct"] = df_cleaned["MntMeatProducts"]/df_cleaned['total_Mnt']
df_cleaned["MntFishProducts_pct"] = df_cleaned["MntFishProducts"]/df_cleaned['total_Mnt']
df_cleaned["MntSweetProducts_pct"] = df_cleaned["MntSweetProducts"]/df_cleaned['total_Mnt']
df_cleaned["MntGoldProds_pct"] = df_cleaned["MntGoldProds"]/df_cleaned['total_Mnt']


In [ ]:
df_cleaned[['MntWines_pct', 'MntFruits_pct', 'MntMeatProducts_pct', 'MntFishProducts_pct', 'MntSweetProducts_pct', 'MntGoldProds_pct']].describe()

In [ ]:
# Create AcceptedCmps to count number of offers each customer accepted
df_cleaned["AcceptedCmps"] = df_cleaned[["AcceptedCmp1","AcceptedCmp2","AcceptedCmp3","AcceptedCmp4", "AcceptedCmp5"]].sum(axis = 1)

In [ ]:
df_cleaned_for_DS = df_cleaned.copy()

## Start to Create Random Forest

In [ ]:
#Balanced dataset by oversampling
count_response_0, count_response_1 = df_cleaned_for_DS.Response.value_counts()
print(count_response_0)
print(count_response_1)

df_cleaned_for_DS_0 = df_cleaned_for_DS[df_cleaned_for_DS["Response"] == 0]
df_cleaned_for_DS_1 = df_cleaned_for_DS[df_cleaned_for_DS["Response"] == 1]

df_cleaned_for_DS_1_over = df_cleaned_for_DS_1.sample(n = count_response_0, replace = True)
df_cleaned_for_DS_1_over.shape

In [ ]:
df_cleaned_for_DS_resampling = pd.concat([df_cleaned_for_DS_1_over, df_cleaned_for_DS_0], axis = 0)

In [ ]:
# split the train and test dataset

X = df_cleaned_for_DS_resampling[feature]
y = df_cleaned_for_DS_resampling.Response

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.4, random_state = 123)

In [ ]:
feature = ["2n Cycle", "Basic", "Graduation", "Master", "PhD", "Absurd", "Alone", "Divorced", "Married", "Single",
            "Together", "Widow", "YOLO", "Income", "Kidhome", "Teenhome", "Recency", "MntWines_pct", "MntFruits_pct",
            "MntMeatProducts_pct", "MntFishProducts_pct", "MntSweetProducts_pct", "MntGoldProds_pct", "NumDealsPurchases",
            "NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases", "NumWebVisitsMonth",
            "AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5",
            "AcceptedCmps", "Complain",
            "account_age", "Age"] 

In [ ]:
from sklearn.model_selection import GridSearchCV

## Use GridSearchCV to find the optimized tree depth

In [ ]:
rfc = RandomForestClassifier(n_estimators=30,
                             random_state=1)

max_depth_range = range(1,16)
param_grid = dict(max_depth=max_depth_range)

grid = GridSearchCV(rfc,
                    param_grid,
                    cv = 10,
                    scoring = 'accuracy')

grid.fit(X_train, y_train)


grid.cv_results_

In [ ]:
grid_mean_scores = grid.cv_results_["mean_test_score"]
grid_test_mean_scores = metrics
# plot the results
sns.mpl.pyplot.plot(max_depth_range, grid_mean_scores)
sns.mpl.pyplot.xlabel('max_depth')
sns.mpl.pyplot.ylabel('Cross-Validated Mean Train Set Accuracy');

In [ ]:
best_rfc = RandomForestClassifier(n_estimators=50,
                                  random_state=1,
                                  max_depth = 12)
best_rfc.fit(X_train, y_train)

rfc_pred = best_rfc.predict(X_test)
accuracy_train = metrics.accuracy_score(y_train, best_rfc.predict(X_train))
accuracy_test = metrics.accuracy_score(y_test, rfc_pred)
print("Accuracy of Random Forest train is: ", accuracy_train)
print("Accuracy of Random Forest test is: ", accuracy_test)

In [ ]:
# compute feature importances
pd.DataFrame({'feature':feature, 'importance':best_rfc.feature_importances_}).sort_values(by='importance',ascending=False)

In [ ]:
DecisionTree_cross_val = DecisionTreeClassifier(max_depth=10, random_state=10)
DecisionTree_cross_val.fit(X_train, y_train)
cross_val_scores = cross_val_score(DecisionTree_cross_val, X_train, y_train, cv = 10)
print("10-fold accuracies:\n",cross_val_scores)
print("10-fold mean accuracy:\n",cross_val_scores.mean())

In [ ]:
y_test_pred_rcf = best_rfc.predict(X_test)

In [ ]:
print("Classification Report:\n", metrics.classification_report(y_test,y_test_pred_rcf))

In [ ]:
pred_probs_rcf =  best_rfc.predict_proba(X_test)
pred_probs_rcf

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_probs_rcf[:,1])

In [ ]:
sns.mpl.pyplot.plot(fpr, tpr,label="DS")
sns.mpl.pyplot.xlim([0, 1])
sns.mpl.pyplot.ylim([0, 1.05])
sns.mpl.pyplot.legend(loc="lower right")
sns.mpl.pyplot.xlabel('False Positive Rate (1 - Specificity)')
sns.mpl.pyplot.ylabel('True Positive Rate (Sensitivity)')

In [ ]:
print("Test AUC: ",metrics.roc_auc_score(y_test, best_rfc.predict(X_test)))